# DNBLab Jupyter Notebook Tutorial 

## Titelanalyse, TOC-Volltextsuche und Ergebnisvisualisierung

Dieses DNBLab Tutorial besteht aus mehreren Teilen und dient als Einführung in die Titelanalyse sowie die Volltextsuche mit Python. Die Titelsuchen werden dabei anhand bibliothekarischer Metadaten durchgeführt, die im Format MARC21-XML vorliegen. Die Volltextsuchen werden anschließend anhand digitalisierter Inhaltsverzeichnisse (kurz TOC für "table of content") bearbeitet. Im letzten Teil des Tutorials werden die Ergebnisse dieser Analysen visualisiert.     

Das Tutorial wird exemplarisch anhand eines Szenarios durchgeführt, in dem wir uns für Hochschulschriften zum Thema "Klimawandel" interessieren, die in der Sachgruppe "Wirtschaft" publiziert wurden. 


**Inhaltsverzeichnis:** 

* [Einrichten der Arbeitsumgebung](#Teil1)
* [Tutorial 1 - Titelanalyse](#Teil2)
* [Tutorial 2 - OC-Volltextsuche]#(#Teil3)
* [Tutorial 3 - Ergebnisvisualisierung]#(#Teil4)



### 1. Einrichten der Arbeitsumgebung - Let's get started  <a class="anchor" id="Teil1"></a>

Um die Arbeitsumgebung für die folgenden Schritte passend einzurichten, sollten zunächst die benötigten Python-Biblitoheken importiert werden. Wir laden dazu ElementTree (als ET), um unser MARC21-XML analysieren zu können sowie Pandas (als pd) zur Datenmanipulation und -analyse.:  

In [1]:
# Laden benötigter Python-Bibliotheken

import xml.etree.ElementTree as ET
import pandas as pd


Im nächsten Schritt laden wir die MARC21-XML-Datei, die wir im folgenden analysieren wollen, in ElementTree und suchen zunächst nach dem Rootverzeichnis. Da uns außerdem interessiert, wieviele verschiedenene Einträge sich überhaupt in der XML-Datei befinden, lassen wir uns dies anzeigen:    


In [2]:
#tree = ET.parse('downloads/data/DNB_freie_hochschulschriften.xml')
tree = ET.parse('downloads/data/dnb-datashop_abzug.xml')  # Laden der MARC-Datei in ElementTree 
root = tree.getroot()                                     # Laden des Root-Verzeichnisses des XML

records = list(root)
total_records = len(records)  

#Ausgabe der Gesamtzahl der in der Datei vorhandenen Records:
print("Gesamtzahl bibliographischer Einträge: ",total_records)



Gesamtzahl bibliographischer Einträge:  3455


Wir wissen nun also, wieviele Titeleinträge sich insgesamt in der Datei befinden und haben schon einmal alles für die weitere Analyse vorbereitet. 

### 2. Ausgabe von Titeln, die ein bestimmtes Suchwort enthalten: 

Die für unsere Fragestellung interessanten Daten befinden sich in der XML-Datei als Inhalt zwischen bestimmten Tags. Wir brauchen also zunächst eine Möglichkeit, diesen Inhalt in Textform aus dem XML zu "holen". 

Da wir zwar nur in den Titeltexten nach unserem Schlüsselwort suchen wollen, als Ergebnis aber möglichst nicht nur den Titel selbst, sondern auch Angaben wir Autor, Publikationsdatum, ISBN etc. wünschen, schreiben wir uns eine allgemeine Funktion für unser XML, um an den Inhalt der unterschiedlichen Felder gelangen zu können. 


Zur Erinnerung: Unser XML ist folgendermaßen aufgabaut: 
```
<?xml version="1.0" encoding="UTF-8"?><collection xmlns="http://www.loc.gov/MARC21/slim">
    <record type="Bibliographic">
        <leader> ... </leader>
        <controlfield> ... </controlfield>
        ...
        <controlfield> ... </controlfield>
        <datafield tag="123" ... >
            <subfield code="x"> ... </subfield>
            ...
            <subfield code="x"> ... </subfield>
        </datafield>
        <datafield tag="456" ...> 
        .... 
```         
Für die Lokalisierung der gewünschen Informationen müssen wir nun natürlich wissen, in welchen Feldern sich welche Informationen befinden -> dies kann bei Bedarf in der MARC-Dokumentation (LINK LINK LINK) nachgeschaut werden. 

Anhand der tag-Information innerhalb der Datafield-tags wird also das entsprechende Feld identifiziert: Das <datafield> mit dem tag="245" steht dabei für den Titeleintrag, im Unterfeld "a" steht dabei der Haupttitel, in ```b``` der Untitel, sofern vorhanden und im Unterfeld "c" wird der oder die für die Publikation Verantwortliche angegeben - dabei kann es sich sowohl um eine Person als auch eine Körperschaft handeln. 
    
Die für uns relevanten Informationen befinden sich also zumeist zwischen bestimmten <subfield>-Tags, die ihrerseits wiederrum anhand der "tag"-Information innerhalb des jeweils übergeordneten <datafield>-Tags identifiziert werden können. In unserem Fall wäre dies beispielhaft für den Haupttitel das <datafield> mit dem tag 245, innerhalb dessen wir den Haupttitel im '''<subfield>''' mit dem code "a" finden.
    
Wir schreiben uns deshalb eine Funktion, die uns den Inhalt aus den entsprechenden \< subfields> holt und nennen sie entsprechend "get_subfield_text". Die Funktion wird mehrere Parameter brauchen, um die unterschiedlichen Felder anzusprechen. Da wir diese allgemein halten wollen, nennen wir die Variablen, die später die jeweiligen Parameter übergeben bekommen, "element", "tag" und "subfield":    
          

In [3]:
def get_subfield_text(element, tag, subfield):
    if element.attrib['tag'] == tag:
            for subelement in element:
                if subelement.attrib['code'] == subfield:
                    return subelement.text
    return None
    


Diese Funktion bekommt später als "element" eine passende XML-node übergeben. Als "tag" wollen wir das jeweiilige MARC-Tag übergeben, welches spezifieren wird, wonach wir genau suchen: Um nach einem Titel zu suchen, wollen wir also bspw. alle Datafield-Tags nach dem "tag" "245" durchsuchen. Wenn wir dagegen nach Autoren suchen wollen, ändern wir die Variable "tag" entsprechend auf "100" usw. Die Variable "subfield" steht, wie der Name sagt, für das jeweilige Subfield, in dem sich die Information befindet: Für unser Titelbeispiel also das Subfield "a" - würden wir hier nur den Untertitel extrahieren wollen, würden wir dagegen im Subfield "b" suchen. 

Mit dieser Funktion haben wir also die Möglichkeit, je nach Bedarf den Text aus den unterschiedlichen Subfields ausgeben zu lassen. :) 


Im weiteren Verlauf legen wir nun ein Keyword fest, nachdem wir suchen möchten. Danach schreiben wir eine Schleife, die durch alle entsprechenden XML-nodes läuft und dabei unsere Funktion aufruft und den Titeltext übergibt. Daraufhin fragen wir, ob unser Keyword im Titeltext enthalten ist. Wenn dies der Fall ist, schreiben wir den Titel in eine Liste namens "Titelsammlung", die wir zuvor definiert haben und erweitern diese Liste, wenn ein weiterer Treffer gefunden wird: 


In [4]:
keyword = "Analyse"     # Hier legen wir unser Suchwort fest - dieses kann natürlich geändert werden
titelsammlung = []      # Definition unser Listenvarialbe für die Titelsammlung 


for record in records:  
        
    for child in record.findall('{http://www.loc.gov/MARC21/slim}datafield'):             
        title = get_subfield_text(child, "245", "a") # Aufruf der oben geschriebenen Funktion, Übergabe der Parameter
        if title:
            if keyword in title:
                titelsammlung.append(title)
              


Um zu sehen, welche Titel unser Code nun gefunden hat, gibt es verschiedene Möglichkeiten: Die einfachste ist sicherlich, einfach unsere Liste über den print-Befehl anzeigen zu lassen: 

In [5]:
print(titelsammlung)

['Quantitative Analyse der Untergrundwirtschaft mit einem linearen Strukturmodell', 'Fotoromane - Analyse eines Massenmediums', 'Analyse der Stilentwicklung in politischen Diskursen während der Französischen Revolution (1789 - 1794)', 'Legitimation einer staatlichen Industriepolitik und Analyse der europäischen Industriepolitik in ausgewählten Teilbereichen', 'Analyse der Entwicklung der pflanzlichen Produktion in der Provinz Zhejiang der VR China unter besonderer Berücksichtigung der agrarpolitischen Veränderungen', 'Analyse heutiger Rezeptionsbedingungen europäischer Erfolgskomödien aus dem 18. Jahrhundert, besonders des "Jeu de l\'amour et du hasard" von Marivaux', 'Analyse und Simulation des sektoralen Wandels der Beschäftigtenstruktur in den Bundesländern der Bundesrepublik Deutschland 1960 - 1990', 'Historische und soziale Analyse der Revolution bei Leo Trotzki', '\x98Die\x9c Analyse der Innovationstätigkeit deutscher Automobilhersteller auf dem Markt für Personenkraf

Alternativ können wir aber bspw. auch ein Dataframe erstellen lassen: Diese Option bietet uns die Programmbibliothek "Pandas" für Python, die wir beim Einrichten unserer Arbeitsumgebung bereits importiert haben - "Pandas" selbst steht dabei für die "Python Data Analysis Library". Ein Pandas-Dataframe kann man sich am besten wie eine Tabelle bzw. einfache Datenbank vorstellen. 

Wir erstellen dazu nun exemplarisch ein Set, welches wir "result" nennen. Da wir bislang nur mit einer Ausgabe-Variable arbeiten, wäre dies noch nicht zwingend nötig, wird aber im Anschluss sinnvoll sein. In ein Set können Objekte beliebigen Datentyps  gespeichert werden. Dieses Set überführen wir im Anschluss in ein Dataframe und lassen uns zuletzt mit dem Befehl "df.head()" die ersten 5 Zeilen des Dataframe anzeigen: 


In [6]:
result = {'title' : titelsammlung}
df = pd.DataFrame(result)

#Gibt die ersten 5 Zeilen des Dataframes aus (zur Kontrolle):
df.head() 


,title
0,Quantitative Analyse der Untergrundwirtschaft ...
1,Fotoromane - Analyse eines Massenmediums
2,Analyse der Stilentwicklung in politischen Dis...
3,Legitimation einer staatlichen Industriepoliti...
4,Analyse der Entwicklung der pflanzlichen Produ...


Alternativ können wir uns auch die letzten 5 Zeilen anzeigen lassen oder das Dataframe komplett als CSV exportieren. Um den Export vorzunehmen, sollte im folgenden Code-Snippet allerdings der Befehl df.tail() "auskommentiert" werden (also einfach eine # davorsetzen, so dass der Code als Kommentar behandelt und nicht ausgeführt wird). Im Gegenzug muss die # vor der letzten Zeile "df.to_csv" natürlich entfernt werden:  

In [8]:
df.tail() # Zeigt die letzten 5 Zeilen des Dataframes an

#Exportieren des gesamten Dataframe als CSV-Datei: 
#df.to_csv("Titeltest.csv", index=False) 

,title
8,Die Analyse der Innovationstätigkeit deutsc...
9,Pädagogische Analyse und Bewertung des Freize...
10,Historische Analyse und Entwicklungslinien der...
11,Zur regulationstheoretischen Analyse der kapit...
12,Messung und Analyse der totalen Faktorprodukti...
